In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration TABD project") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()

In [4]:
spark.sql(
    """
    SHOW DATABASES
    """
).show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [3]:
spark.sql(
    """
    DROP DATABASE IF EXISTS project_tabd CASCADE
    """
)

DataFrame[]

In [5]:
# you can choose any location in HDFS, just be organized 
# Your data lake will grow with time and will become a swamp
spark.sql(
    """
    CREATE DATABASE project_tabd LOCATION 'hdfs://hdfs-nn:9000/warehouse/tabd.db/'
    """
)

DataFrame[]

In [6]:
spark.sql(
    """
    SHOW DATABASES
    """
).show()

+------------+
|   namespace|
+------------+
|     default|
|project_tabd|
+------------+



In [7]:
spark.sql(
    """
    SHOW TABLES FROM project_tabd
    """
).show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+



In [9]:
spark.sql(
    """
    DROP TABLE IF EXISTS project_tabd.parquet_table_weather
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE project_tabd.parquet_table_weather (
        date DATE,
        month INT,
        day INT,
        station_id VARCHAR(15),
        station_name VARCHAR(100),
        latitude DOUBLE,
        longitude DOUBLE,
        air_temperature_avg DOUBLE,
        air_temperature_min DOUBLE,
        air_temperature_max DOUBLE,
        global_radiation_total DOUBLE,
        rain_precipitation_qty DOUBLE        
               
    )
    STORED AS PARQUET
    PARTITIONED BY (
        year INT
    )
    LOCATION 'hdfs://hdfs-nn:9000/warehouse/tabd.db/parquet_table_weather/'
    """
)

DataFrame[]

In [10]:
spark.sql(
    """
    DESCRIBE FORMATTED project_tabd.parquet_table_weather
    """
).toPandas() 

,col_name,data_type,comment
0,date,date,None
1,month,int,None
2,day,int,None
3,station_id,varchar(15),None
4,station_name,varchar(100),None
5,latitude,double,None
6,longitude,double,None
7,air_temperature_avg,double,None
8,air_temperature_min,double,None
9,air_temperature_max,double,None


In [11]:
spark.sql(
    """
    SHOW TABLES FROM project_tabd
    """
).show()

+------------+--------------------+-----------+
|    database|           tableName|isTemporary|
+------------+--------------------+-----------+
|project_tabd|parquet_table_wea...|      false|
+------------+--------------------+-----------+



In [14]:
# recover partitions is needed so that the Hive Metastore (Catalog)
# is updated. Otherwise Hive and the querying engines do not know
# that there are new parittions in the partitioned table.
spark.catalog.recoverPartitions("project_tabd.parquet_table_weather")

spark.sql(
    """
    SELECT *
    FROM project_tabd.parquet_table_weather
    """
).show()


+----------+-----+---+----------+--------------------+-----------+-----------+-------------------+-------------------+-------------------+----------------------+----------------------+----+
|      date|month|day|station_id|        station_name|   latitude|  longitude|air_temperature_avg|air_temperature_min|air_temperature_max|global_radiation_total|rain_precipitation_qty|year|
+----------+-----+---+----------+--------------------+-----------+-----------+-------------------+-------------------+-------------------+----------------------+----------------------+----+
|2006-01-01|    1|  1|   1200545|Porto - Pedras Ru...|41.23350278|-8.68133333|               11.1|                6.9|               14.3|                6267.1|                   0.7|2006|
|2006-01-01|    1|  1|   1200548| Coimbra (Aeródromo)|    40.1576|-8.46851667|               10.0|                8.6|               12.7|                5513.9|                   2.3|2006|
|2006-01-01|    1|  1|   1200551|Viana do Castelo 

In [21]:
spark.sql(
    """
    SELECT air_temperature_max, date,station_id,station_name
    FROM project_tabd.parquet_table_weather
    WHERE station_id="1200545" and year=2000
    """
).toPandas()

,air_temperature_max,date,station_id,station_name
0,10.8,2000-01-01,1200545,Porto - Pedras Rubras
1,11.5,2000-01-02,1200545,Porto - Pedras Rubras
2,11.9,2000-01-03,1200545,Porto - Pedras Rubras
3,12.1,2000-01-04,1200545,Porto - Pedras Rubras
4,13.4,2000-01-05,1200545,Porto - Pedras Rubras
...,...,...,...,...
361,13.1,2000-12-27,1200545,Porto - Pedras Rubras
362,15.5,2000-12-28,1200545,Porto - Pedras Rubras
363,-990.0,2000-12-29,1200545,Porto - Pedras Rubras
364,-990.0,2000-12-30,1200545,Porto - Pedras Rubras


In [12]:
# Let's look into HDFS

In [13]:
# Let's put the files into HDFS

In [14]:
spark.stop()